# Generating Account Features and Exploring Clustering Techniques

## Step 1. Generating Account Features

Run a bigquery to get data between '2019-12-1' and '2019-12-31'

where  
    - TransactionResult="tesSUCCESS"

Get BigQuery Data into workable form as show below:


|Account Hash|Sum of XRP Received| Sum of XRP Sent|Sum of Fees| Number of Txns|
|-----|-------|------|------|-----|




Importing Packages

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
#from scipy.stats import shapiro
from scipy.cluster.hierarchy import dendrogram, linkage 
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from google.cloud import bigquery
from google.oauth2 import service_account


In [28]:
import sys
from os import path

In [29]:
from ../../Utils/xrp_ml_utils.py import *

SyntaxError: invalid syntax (<ipython-input-29-e56ad4161906>, line 1)

In [30]:
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('../../Utils/xrp_ml_utils.py'),os.path.pardir)))

In [31]:
import xrp_ml_utils.py

ImportError: No module named 'xrp_ml_utils'

Setting Window for Query

In [5]:
start_date='2019-12-1'
end_date='2019-12-31'

In [64]:
cred_path='../../credential/Xpring Dev Sandbox-edb2a98acf17.json'

In [65]:
def get_gcp_creds(key_path):
    """
    Return GCP credentials from local .json
    
    Parameters
    ----------
    key_path: str
        Path to local credential .json
    """
    try:
        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
        )
    except:
        credentials = None
    return credentials;

In [71]:
def gbq_query(query,cred, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(credentials=cred)
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [76]:
#Query and query_parameters
query1 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"

AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp

LIMIT 1000000

"""
query_params1 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]



In [77]:
xrp1=gbq_query(query1,get_gcp_creds(cred_path),query_params1)

In [78]:
xrp1

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
0,2019-12-01 00:00:00+00:00,rDcPjwP7aAtLAPUWMJoEt3ZWS2xjqnEkW6,rPGUevcGnMoF85sfZrtL1x6EXhM2wG4Fmx,12,3045022100C302E68D787E67F918E952228482AFA436AC...,NaN,51752122,Payment
1,2019-12-01 00:00:00+00:00,rQNHoQJNBoRSN8U16ufcqrFDYwYCgqguQY,rKBMcNV8xwVHvw9nNk3WCZtnnXuGiggaoe,7632,304402207D60892A0B15AFB756959B5E3D49B612506FAF...,NaN,51752122,Payment
2,2019-12-01 00:00:00+00:00,rpnScMMLsTdFofHpLwTFrUAT5F4kSKggiq,rKoyASc15cjD6sFQs4w9nvX3GRVXkAHt4t,12,3044022024EBE8941EBE1C183EB258D82639BFEC396C84...,NaN,51752122,Payment
3,2019-12-01 00:00:00+00:00,rDc2KgoWg2CQaHPUDmmpqF8Xd7xQzzrqbN,rHo16GoWkEZar8zhEDfGxcRxMLo8qMqk1b,12,30440220025712E85B2F22C95F14D0EBA4DC031BDB3D74...,NaN,51752122,Payment
4,2019-12-01 00:00:00+00:00,r4GXbgT8eekMR5FEapdMaiJikyNTw7Pf45,rB2UYkwenKDKxCFCXkx17fyqWw3ebYWqjE,12,3045022100E1F584F79C539C17B24C63A2D6C487B92713...,NaN,51752122,Payment
...,...,...,...,...,...,...,...,...
999995,2019-12-01 05:48:41+00:00,rh3VLyj1GbQjX7eA15BwUagEhSrPHmLkSR,None,12,304402200CABD23D64CBC418C63C3744B1DDA517002CB1...,NaN,51757334,OfferCreate
999996,2019-12-01 05:48:41+00:00,rh3VLyj1GbQjX7eA15BwUagEhSrPHmLkSR,None,12,3045022100E7F6C9F34135FED935263ED4AF5C1BFA17C8...,NaN,51757334,OfferCreate
999997,2019-12-01 05:48:41+00:00,rBXqvQZw3DoioL1mP5A7ojVs48ZvYdMxqD,rstaazHNsnJWtwA2xFxjBWWFip67c9Ky5P,12,3045022100E2396F9F0F85534535DA7BC318444B95F194...,NaN,51757334,Payment
999998,2019-12-01 05:48:41+00:00,rh3VLyj1GbQjX7eA15BwUagEhSrPHmLkSR,None,12,3045022100ECD748A2B260CACC2FF2BF1A3927073AF7AF...,NaN,51757334,OfferCreate


In [79]:
xrp1.isnull().sum()

TimeStamp               0
Account                 0
Destination        295596
Fee                     0
TxnSignature          367
AmountXRP          991520
LedgerIndex             0
TransactionType         0
dtype: int64

In [80]:
xrp1['TransactionType'].value_counts()

Payment         704404
OfferCreate     288214
OfferCancel       7220
AccountSet         152
TrustSet             7
EscrowFinish         3
Name: TransactionType, dtype: int64

In [81]:
#Query and query_parameters
query2 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"
AND TxnSignature IS NOT NULL
AND Destination IS NOT NULL
AND AmountXRP  IS NOT NULL
AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp

LIMIT 1000000

"""
query_params2 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]




In [84]:
path='../../Data/'

In [85]:
xrp2=gbq_query(query2,get_gcp_creds(cred_path),query_params2)



KeyboardInterrupt: 

In [83]:
xrp2

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
0,2019-12-01 00:00:01+00:00,rfQLYTZk7iBSekEv1GughC1ruWfCDeZXVz,rP1afBEfikTz7hJh2ExCDni9W4Bx1dUMRk,16305,30450221009B3E7024448E56F1E580F4AFCBBC9BECF050...,222000000,51752123,Payment
1,2019-12-01 00:00:01+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12645,3045022100C1839ADD174EF85C785E452A83CF0AD655BF...,32,51752123,Payment
2,2019-12-01 00:00:02+00:00,rw2ciyaNshpHe7bCHo4bRWq6pqqynnWKQg,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,40,3045022100BE2F8FBCC8A76A31CDA4FF0A13BA2120EB71...,360980050,51752124,Payment
3,2019-12-01 00:00:10+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,3045022100D41EF19727BECD8B77976FC731F86A6F4CD1...,30,51752125,Payment
4,2019-12-01 00:00:11+00:00,rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,12,3045022100B38D74883970A9A21497A520BA4A56C28EF0...,30,51752126,Payment
...,...,...,...,...,...,...,...,...
707188,2019-12-30 23:59:42+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,304402204918F22DEB8B151FA28FA64AA994B1392B3320...,325686594,52409894,Payment
707189,2019-12-30 23:59:42+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,304502210087CC7A6B27090ED095E8BA8F479EF628695C...,275267296,52409894,Payment
707190,2019-12-30 23:59:50+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,304402200A23218923346C7E5C868B6941700630A6FBF6...,5083632552,52409895,Payment
707191,2019-12-30 23:59:50+00:00,rDsbeomae4FXwgQTJp9Rs64Qg9vDiTCdBv,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,60000,3044022075F91DAF97CC80563D673B9DFB742DCF73F782...,732509247,52409895,Payment


In [86]:
xrp2.to_csv(os.path.join(path,r'12_1_2019_to_12_30_2019_DownselectALL.csv'))

In [ ]:

def dict_accounthash(data):
    #Creating a merged a numpy array of unique account and destination hashes
    UniqueAccount=np.unique(data['Account'])
    UniqueDestination=np.unique(data['Destination'])
    UniqueHash=np.unique(np.concatenate([UniqueDestination, UniqueAccount]))
    #Creating a dictionary with account hashes as keys and account balances initialized at 0 as the values
    #The value is an array of length 3 for summing xrp received, xrp sent, and sum of fees
    UniqueHashList=UniqueHash.tolist()
    listofzeros = [[0] *3 ]*len(UniqueHashList)
    dicAccBal = {k:v for k,v in zip(UniqueHashList,listofzeros)}
    

        #iterate through each transaction
    for index, row in xrp.iterrows():
        #Checking if the transaction's timestamp is within time window specified by starttime and endtime
        if (row['TimeStamp'] < endtime) & (row['TimeStamp'] > starttime):
        #Adding account balances to respective dictionary balance
            dicAccBal[row['Account']][i] += (-1) * (row['AmountXRP'])
            dicAccBal[row['Destination']][i] += (1) * (row['AmountXRP'])
    
    
    return dicAccBal



    
